In [22]:
import os

import yaml
from config import SETTINGS
from pathlib import Path
import pandas as pd

import seabeepy as sb
import requests

# Refactor `config.seabee.yaml` for niva-tidy missions

In [23]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

In [24]:
# Parent directories containing flight folders to process
base_dirs = [
    Path(r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022"),
    Path(r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2023"),
]

In [25]:
def get_spectrum_type(spec: str):
    if spec is None:
        return None
    if spec.lower() in ["rgb", "msi", "hsi"]:
        return spec.lower()
    if spec.lower() == "ms":
        return "msi"


def template_config() -> dict:
    return dict(
        grouping=None,
        area=None,
        datetime=None,
        spectrum_type=None,
        elevation=None,
        project=None,
        nfiles=None,
        organisation=None,
        creator_name=None,
        theme="habitat",
        mosaic=False,
        publish=True,
        classify=True,
    )


def niva_name_to_config(name: str):

    parts = name.split("_")

    if len(parts) == 7:
        org, date, group, area, spect, elev, _ = parts
    elif len(parts) == 6:
        org, date, group, area, spect, elev = parts
    elif len(parts) == 5:
        org, date, group, area, spect = parts
        elev = None
    elif len(parts) == 4:
        org, date, group, area = parts
        spect = None
        elev = None
    return dict(
        grouping=group,
        area=area,
        datetime=date,
        spectrum_type=get_spectrum_type(spect),
        organisation=org,
        elevation=int(elev) if elev is not None and elev.isnumeric() else elev,
    )


def parse_config(dir_path: Path):
    conf = {}
    config_path = dir_path / "config.seabee.yaml"
    if config_path.exists():
        with open(config_path, "r") as f:
            conf = yaml.safe_load(f)
    return conf


def write_new_config(path: Path, data):
    with open(path, "w") as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False, allow_unicode=True)


def merge_conf(folder_config: dict, existing_config: dict) -> dict:
    """Merge the folder_config with the existing_config

    Use various rules some times use folder name is best, other times the config
    """

    new_config = template_config()
    for key, val in folder_config.items():
        new_config[key] = val
    for key in ["creator_name", "project", "mosaic", "publish", "theme", "area", "spectrum_type"]:
        if key in existing_config:
            if key == "creator_name" and "_" in existing_config[key]:
                new_config[key] = " ".join(existing_config[key].split("_"))
            else:
                new_config[key] = existing_config[key]
    
    # If file config is placeholder switch back to config based on folder name
    if new_config["area"].lower() in ["site", "oslo"]:
        new_config["area"] = folder_config["area"]

    if new_config["spectrum_type"] is None:
        new_config["spectrum_type"] = existing_config["spectrum_type"]
    new_config["folder_grouping"] = folder_config["grouping"]
    return apply_rules(new_config)


def apply_rules(config: dict, folder_name=""):
    # msi -> ms + lower
    config["spectrum_type"] = get_spectrum_type(config["spectrum_type"])
    if not isinstance(config["elevation"], int):
        config["elevation"] = None
    # No underscore in attributes, this destroys the layer name
    if "_" in config["area"]:
        config["area"] = config["area"].replace("_", "-")
    
    if "oldberg" in config["area"]:
        config["area"] = config["area"].replace("oldberg", "olberg")
    
    if config["area"].startswith("larvik-"):
        config["area"] = config["area"].split("-", 1)[-1]

    if config["area"].startswith("runde-"):
        config["area"] = config["area"].split("-", 1)[-1]

    if config["project"] is None:
        # Around three missions with missing project, use seabee
        config["project"] = "seabee"
    
    config["project"] = config["project"].lower()
    
    if config["project"] in ["runderunderunderunde", "srunde"]:
        config["project"] = "runde"
    
    if config["area"] == "runde":
        config["project"] = "runde"

    if config["area"].startswith("runde-"):
        config["area"] = config["area"].split("-", 1)[-1]
    
    if folder_name: 
        if config["area"] == "olberg":
            config["area"] = niva_name_to_config(folder_name)["area"]
        if "_msi_" in folder_name:
             config["spectrum_type"]="msi"
        elif "_hsi_" in folder_name:
            config["spectrum_type"]="hsi"
        elif "_rgb_" in folder_name:
            config["spectrum_type"]="rgb" 
    # Check folder grouping
    if config["project"] in ["seabee"]:
        # try to use naming from folder name for project
        config["project"] = config["folder_grouping"]
    
    if config["project"] in ["oslo", "io"]:
        config["project"] = "io23"

    if config["project"] == "io23" and "oslo" not in config["area"]:
        config["area"] = "oslo-" + config["area"]

    if config["project"] == "plastinoland":
        config["project"] = "plastnoland"

    if config["project"] == "larvik":
        # spresial rule for larvik ending up as project?
        config["project"] = "zosmap"
        

    if config["organisation"] in ["spectorfly"]:
        config["organisation"] = "spectrofly"
    
    if "stege-nor" in config["area"].lower():
        config["grouping"] = f"{config['organisation'].lower()}"
        config["area"] = config["area"].lower()
        if config["area"].lower() == "stege-nor" :
            config["area"] = "stege-nor-full"
    elif folder_name.endswith("_odm"):
        config["grouping"] = f"{config['organisation'].lower()}-{config['project'].lower()}-odm-test"
    else:
        config["grouping"] = f"{config['organisation'].lower()}-{config['project'].lower()}"

    return config


def to_seabee_config(df: pd.DataFrame, tmp_dir):
    """Try to convert the dataframe to seabee config files

    Try to save df to seabee config files and resturn a new dataframe with the results
    this way we can play a bit with various changes and resulting layer name. The config files
    are saved in tmp_dir.
    """
    mission_dict = dict(folder_name=[], layer_name=[], valid=[], dir_path=[])
    keys = template_config().keys()
    mission_dict.update({k: [] for k in keys})
    mission_dict["folder_grouping"] = []
    mission_dict["old_layer_name"] = []
    for _, data in df.iterrows():
        mission_name = data["folder_name"]
        config = apply_rules(data[list(keys) + ["folder_grouping"]].to_dict(), mission_name)
        clean_config = {k: config[k] for k in keys if config[k] is not None}
        nfiles = len(sb.ortho.list_images(data.dir_path / "images", verbose=False))
        mission_dict["folder_name"].append(mission_name)
        try:
            sb.ortho.CONFIG_SCHEMA.validate({**clean_config, "nfiles": 1})
            write_new_config(tmp_dir / "config.seabee.yaml", clean_config)
            layer_name = sb.ortho.get_layer_name(tmp_dir)
        except Exception as e:
            print(f"Error in {mission_name}: {e}")
            config["nfiles"] = nfiles
            mission_dict["layer_name"].append("error")
            mission_dict["valid"].append(False)
        else:
            config["nfiles"] = nfiles
            mission_dict["layer_name"].append(layer_name)
            mission_dict["valid"].append(True)
            tmp_file = tmp_dir / f"{mission_name}.yaml"
            write_new_config(tmp_file, config)

        for k, v in config.items():
            mission_dict[k].append(v)
        mission_dict["dir_path"].append(data.dir_path)
        mission_dict["old_layer_name"].append(data.old_layer_name)
        

    return pd.DataFrame(mission_dict).astype(dict(elevation="Int16"))

In [26]:
dir_list = [
    p
    for p in base_dirs
    for p in p.iterdir()
    if p.is_dir() and (p / "config.seabee.yaml").exists()
]

In [27]:
tmp_dir = Path("./tmp")
tmp_dir.mkdir(exist_ok=True)

In [28]:
mission_dict = dict(folder_name=[], dir_path=[])
mission_dict.update({k: [] for k in template_config().keys()})
mission_dict["folder_grouping"] = []
mission_dict["old_layer_name"] = []
for dir_path in dir_list:
    # Update config
    mission_name = os.path.split(dir_path)[-1]
    existing_conf = parse_config(dir_path)
    write_new_config(tmp_dir / "config.seabee.yaml", existing_conf)
    mission_dict["old_layer_name"].append(sb.ortho.get_layer_name(tmp_dir))
    folder_conf = niva_name_to_config(mission_name)
    new_conf = merge_conf(folder_conf, existing_conf)
    mission_dict["folder_name"].append(mission_name)
    nfiles = len(sb.ortho.list_images(dir_path / "images", verbose=False))
    new_conf["nfiles"] = nfiles
    for k, v in new_conf.items():
        mission_dict[k].append(v)
    mission_dict["dir_path"].append(dir_path)

In [29]:
df = pd.DataFrame(mission_dict).astype(dict(elevation="Int16"))
df

,folder_name,dir_path,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish,classify,folder_grouping,old_layer_name
0,niva_202208181236_kelpmap_vega-s_msi_60,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-kelpmap,vega-s,202208181236,msi,60,kelpmap,4272,niva,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_vega-s_20220818_MSI_120m
1,niva_202205180845_plastinoland_drammen_documen...,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-plastnoland,drammen,202205180845,msi,0,plastnoland,112,niva,Medyan Ghareeb,Habitat,False,True,True,plastinoland,Mapping_SITE_YYYYMMDDHHMM_MSI_100m
2,spectrofly_202208180945_kelpmap_vega-s_msi_120,/home/notebook/shared-seabee-ns9879k/niva-tidy...,spectrofly-kelpmap,vega-s,202208180945,msi,120,kelpmap,4850,spectrofly,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_vega-s_202208180945_MSI_120m
3,niva_202208191055_kelpmap_vega-n_msi_120,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-kelpmap,vega-n,202208191055,msi,120,kelpmap,4956,niva,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_SITE_YYYYMMDDHHMM_MSI_100m
4,spectrofly_202208311309_runde_remoy_msi_120,/home/notebook/shared-seabee-ns9879k/niva-tidy...,spectrofly-runde,remoy,202208311309,rgb,120,runde,0,spectrofly,Medyan Ghareeb,Habitat,False,True,True,runde,Mapping_remoy_202208311404_rgb_120m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,niva_202308290947_halden_h29_rgb_80,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-sabicas,halden-h29,202308290947,rgb,80,sabicas,72,niva,Medyan Ghareeb,Habitat,False,True,True,halden,Mapping_halden_h29_202308290947_rgb_80m
184,niva_202310031115_io_husviksbaen_rgb_80,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-io23,oslo-husviksbaen,202310031115,rgb,80,io23,196,niva,Medyan Ghareeb,Habitat,False,True,True,io,Mapping_oslo_202310031115_rgb_90m
185,niva_202308301044_halden_h23_msi_80,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-sabicas,halden-h23,202308301044,msi,80,sabicas,693,niva,Medyan Ghareeb,Habitat,False,True,True,halden,Mapping_halden_h23_202308301044_msi_80m
186,niva_202309271230_ascomap_bringnes_msi_100,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-ascomap,finnmark-bringnes,202309271230,msi,100,ascomap,4116,niva,Medyan Ghareeb,Habitat,False,True,True,ascomap,Mapping_finnmark_bringnes_202309271230_msi_100m


In [30]:
df = to_seabee_config(df, tmp_dir)
df

Error in niva_202209_runde_runde_otter: Key 'datetime' error:
Or(<function <lambda> at 0x7fa86d798040>, <function <lambda> at 0x7fa86d7db640>) did not validate '202209'
<lambda>('202209') raised ValueError("time data '202209' does not match format '%Y%m%d'")
<lambda>('202209') raised ValueError("time data '202209' does not match format '%Y%m%d%H%M'")
Error in niva_yyyymmddhhmm_area_site_typ_alt: Key 'datetime' error:
Or(<function <lambda> at 0x7fa86d798040>, <function <lambda> at 0x7fa86d7db640>) did not validate 'yyyymmddhhmm'
<lambda>('yyyymmddhhmm') raised ValueError("time data 'yyyymmddhhmm' does not match format '%Y%m%d'")
<lambda>('yyyymmddhhmm') raised ValueError("time data 'yyyymmddhhmm' does not match format '%Y%m%d%H%M'")
Error in niva_yyyymmddhhmm_area_site_typ_alt - Copy: Key 'datetime' error:
Or(<function <lambda> at 0x7fa86d798040>, <function <lambda> at 0x7fa86d7db640>) did not validate 'yyyymmddhhmm'
<lambda>('yyyymmddhhmm') raised ValueError("time data 'yyyymmddhhmm' d

,folder_name,layer_name,valid,dir_path,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish,classify,folder_grouping,old_layer_name
0,niva_202208181236_kelpmap_vega-s_msi_60,niva-kelpmap_vega-s_202208181236_msi_60m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-kelpmap,vega-s,202208181236,msi,60,kelpmap,4272,niva,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_vega-s_20220818_MSI_120m
1,niva_202205180845_plastinoland_drammen_documen...,niva-plastnoland_drammen_202205180845_msi,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-plastnoland,drammen,202205180845,msi,0,plastnoland,112,niva,Medyan Ghareeb,Habitat,False,True,True,plastinoland,Mapping_SITE_YYYYMMDDHHMM_MSI_100m
2,spectrofly_202208180945_kelpmap_vega-s_msi_120,spectrofly-kelpmap_vega-s_202208180945_msi_120m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,spectrofly-kelpmap,vega-s,202208180945,msi,120,kelpmap,4850,spectrofly,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_vega-s_202208180945_MSI_120m
3,niva_202208191055_kelpmap_vega-n_msi_120,niva-kelpmap_vega-n_202208191055_msi_120m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-kelpmap,vega-n,202208191055,msi,120,kelpmap,4956,niva,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_SITE_YYYYMMDDHHMM_MSI_100m
4,spectrofly_202208311309_runde_remoy_msi_120,spectrofly-runde_remoy_202208311309_msi_120m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,spectrofly-runde,remoy,202208311309,msi,120,runde,0,spectrofly,Medyan Ghareeb,Habitat,False,True,True,runde,Mapping_remoy_202208311404_rgb_120m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,niva_202308290947_halden_h29_rgb_80,niva-sabicas_halden-h29_202308290947_rgb_80m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-sabicas,halden-h29,202308290947,rgb,80,sabicas,72,niva,Medyan Ghareeb,Habitat,False,True,True,halden,Mapping_halden_h29_202308290947_rgb_80m
184,niva_202310031115_io_husviksbaen_rgb_80,niva-io23_oslo-husviksbaen_202310031115_rgb_80m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-io23,oslo-husviksbaen,202310031115,rgb,80,io23,196,niva,Medyan Ghareeb,Habitat,False,True,True,io,Mapping_oslo_202310031115_rgb_90m
185,niva_202308301044_halden_h23_msi_80,niva-sabicas_halden-h23_202308301044_msi_80m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-sabicas,halden-h23,202308301044,msi,80,sabicas,693,niva,Medyan Ghareeb,Habitat,False,True,True,halden,Mapping_halden_h23_202308301044_msi_80m
186,niva_202309271230_ascomap_bringnes_msi_100,niva-ascomap_finnmark-bringnes_202309271230_ms...,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-ascomap,finnmark-bringnes,202309271230,msi,100,ascomap,4116,niva,Medyan Ghareeb,Habitat,False,True,True,ascomap,Mapping_finnmark_bringnes_202309271230_msi_100m


# Valid config

In [31]:
df[df.valid]

,folder_name,layer_name,valid,dir_path,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish,classify,folder_grouping,old_layer_name
0,niva_202208181236_kelpmap_vega-s_msi_60,niva-kelpmap_vega-s_202208181236_msi_60m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-kelpmap,vega-s,202208181236,msi,60,kelpmap,4272,niva,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_vega-s_20220818_MSI_120m
1,niva_202205180845_plastinoland_drammen_documen...,niva-plastnoland_drammen_202205180845_msi,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-plastnoland,drammen,202205180845,msi,0,plastnoland,112,niva,Medyan Ghareeb,Habitat,False,True,True,plastinoland,Mapping_SITE_YYYYMMDDHHMM_MSI_100m
2,spectrofly_202208180945_kelpmap_vega-s_msi_120,spectrofly-kelpmap_vega-s_202208180945_msi_120m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,spectrofly-kelpmap,vega-s,202208180945,msi,120,kelpmap,4850,spectrofly,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_vega-s_202208180945_MSI_120m
3,niva_202208191055_kelpmap_vega-n_msi_120,niva-kelpmap_vega-n_202208191055_msi_120m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-kelpmap,vega-n,202208191055,msi,120,kelpmap,4956,niva,Medyan Ghareeb,Habitat,False,True,True,kelpmap,Mapping_SITE_YYYYMMDDHHMM_MSI_100m
4,spectrofly_202208311309_runde_remoy_msi_120,spectrofly-runde_remoy_202208311309_msi_120m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,spectrofly-runde,remoy,202208311309,msi,120,runde,0,spectrofly,Medyan Ghareeb,Habitat,False,True,True,runde,Mapping_remoy_202208311404_rgb_120m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,niva_202308290947_halden_h29_rgb_80,niva-sabicas_halden-h29_202308290947_rgb_80m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-sabicas,halden-h29,202308290947,rgb,80,sabicas,72,niva,Medyan Ghareeb,Habitat,False,True,True,halden,Mapping_halden_h29_202308290947_rgb_80m
184,niva_202310031115_io_husviksbaen_rgb_80,niva-io23_oslo-husviksbaen_202310031115_rgb_80m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-io23,oslo-husviksbaen,202310031115,rgb,80,io23,196,niva,Medyan Ghareeb,Habitat,False,True,True,io,Mapping_oslo_202310031115_rgb_90m
185,niva_202308301044_halden_h23_msi_80,niva-sabicas_halden-h23_202308301044_msi_80m,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-sabicas,halden-h23,202308301044,msi,80,sabicas,693,niva,Medyan Ghareeb,Habitat,False,True,True,halden,Mapping_halden_h23_202308301044_msi_80m
186,niva_202309271230_ascomap_bringnes_msi_100,niva-ascomap_finnmark-bringnes_202309271230_ms...,True,/home/notebook/shared-seabee-ns9879k/niva-tidy...,niva-ascomap,finnmark-bringnes,202309271230,msi,100,ascomap,4116,niva,Medyan Ghareeb,Habitat,False,True,True,ascomap,Mapping_finnmark_bringnes_202309271230_msi_100m


# Save the current config

In case something goes wrong keep a local state

In [32]:
keys = [k for k in template_config()]
df[df.valid][["layer_name", "folder_name", "old_layer_name"] + keys].sort_values("layer_name").to_csv("niva-naming-overview.csv", index=False)

# Delete geonode datasets

In [45]:
base_url = sb.geo.GEONODE_URL

auth = (SETTINGS.GEONODE_USER, SETTINGS.GEONODE_PASSWORD)

In [47]:
for _, data in df.iterrows():
    try:
        ds = sb.geo.get_dataset_by_title(data.old_layer_name)
    except:
        pass
    else:
        print(f"Deleting {data.old_layer_name}")
        r = requests.delete(f"{base_url}resources/{ds['pk']}", auth=auth)
        print(r.status_code)

Deleting Mapping_SITE_YYYYMMDDHHMM_MSI_100m
204
Deleting Mapping_vega-s_202208180945_MSI_120m
204
Deleting Mapping_vega-s_202208181238_MSI_120m
204
Deleting Mapping_mfs_202204281257_rgb_90m
204
Deleting Mapping_vega-s_202208181012_MSI_60m
204
Deleting Mapping_landsteilene_202203151133_rgb_80m
204
Deleting Mapping_vega-n_202208190944_MSI_120m
204
Deleting Mapping_vega-s_202208181139_MSI_120m
204
Deleting Mapping_runde_202209021500_rgb_120m
204
Deleting Mapping_remoy_202208311315_msi_60m
204
Deleting Mapping_vega-s_202208181357_MSI_120m
204
Deleting Mapping_runde_202209011441_MSI_120m
204
Deleting Mapping_vega-s_202208180814_rgb_115m
204
Deleting Mapping_remoy_202208310730_rgb_120m
204
Deleting Mapping_remoy_202208310855_MSI_60m
204
Deleting Mapping_vega-n_202208190819_MSI_120m
204
Deleting Mapping_remoy_202208311309_rgb_100m
204
Deleting Mapping_remoy_202208311043_msi_120m
204
Deleting Mapping_vega-n_202208191500_rgb_120m
204
Deleting Mapping_runde_202209010853_MSI_100m
204
Deleting Map